In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
import asyncio
from pathlib import Path
from agents import Agent, Runner, ModelSettings
from agents.tool import function_tool
import fitz
import re
import openai

# ─────────────────────────────────────────────
# 2️⃣ Resume folder (PDFs)
# ─────────────────────────────────────────────
CURRENT_DIR = Path.cwd()
RESUME_DIR = CURRENT_DIR / "resume"   # e.g. Resume Keyword Search Agent/resume

print(f"📂 Using resume folder: {RESUME_DIR}")
if not RESUME_DIR.exists():
    print("⚠️ Warning: resume folder does not exist, but code will still run.")


# ─────────────────────────────────────────────
# 3️⃣ Resume folder scanning (PDF keyword matching)
# ─────────────────────────────────────────────
@function_tool(name_override="scan_resumes_for_keywords")
def scan_resumes_for_keywords(keywords: list[str]) -> list[dict]:
    """
    Searches all PDF resumes in the RESUME_DIR for specific keywords.
    For each match found, a small snippet and page number are recorded.

    Args:
        keywords (list[str]): List of extracted job-related keywords.

    Returns:
        list[dict]: Matching results including:
            - filename: Resume where the keyword was found
            - keyword: The matched keyword
            - match_snippet: First 250 chars around the match
            - page: Page number in the PDF where it appears
    """
    results = []
    for file in RESUME_DIR.glob("*.pdf"):
        doc = fitz.open(file)
        for page in doc:
            text = page.get_text()
            for kw in keywords:
                if kw.lower() in text.lower():
                    results.append({
                        "filename": file.name,
                        "keyword": kw,
                        "match_snippet": text[:250].strip(),
                        "page": page.number + 1
                    })
        doc.close()
    return results



# ─────────────────────────────────────────────
# 4️⃣ Extract keywords from Job Description text
# ─────────────────────────────────────────────

@function_tool
def extract_keywords_from_job_description(job_text: str) -> list[str]:
    """
    Uses GPT to extract 10–15 relevant keywords from a job description.
    These may include required skills, technologies, responsibilities, and tools.

    Args:
        job_text (str): The full job description text.

    Returns:
        list[str]: A refined list of extracted keywords and key phrases.
    """

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "Extract 10–15 key skills, tools, and responsibilities from this job description."
            },
            {"role": "user", "content": job_text}
        ],
        temperature=0.3
    )

    # Format output → split lines returned by the model & clean bullet characters
    lines = response.choices[0].message.content.splitlines()
    return [line.strip("*•- ").strip() for line in lines if line.strip()]


# ─────────────────────────────────────────────
# 5️⃣ Transcribe interview audio file (Whisper)
# ─────────────────────────────────────────────

@function_tool
def transcribe_interview(file_path: str) -> str:
    """
    Converts the interview audio file to text using Whisper.
    The transcription will later be used to evaluate interview relevance.

    Args:
        file_path (str): Path to the audio file (.mp3, .wav, etc.).

    Returns:
        str: Transcribed text from the interview audio.
    """
    
    with open(file_path, "rb") as audio_file:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )

    return transcript.text



# ─────────────────────────────────────────────
# 6️⃣ Analyze interview relevance to job description
# ─────────────────────────────────────────────

@function_tool
def analyze_interview_relevance(interview_text: str, job_description: str) -> str:
    """
    Uses GPT to evaluate how well the interview questions and responses
    align with the job description. Highlights strengths and missing points.

    Args:
        interview_text (str): Transcribed interview text.
        job_description (str): The original job posting or requirements.

    Returns:
        str: Helpful HR-style feedback summarizing alignment quality.
    """

    system_msg = (
        "You are an HR assistant. Evaluate how well the interview questions align "
        "with the job description. Be specific and helpful. Mention which areas "
        "were strong or missing."
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_msg},
            {
                "role": "user",
                "content": f"Interview:\n{interview_text}\n\nJob Description:\n{job_description}"
            }
        ],
        temperature=0.4
    )

    return response.choices[0].message.content.strip()


# ─────────────────────────────────────────────
# 7️⃣ Recruiter AI Agent configuration (multi-tool pipeline)
# ─────────────────────────────────────────────

# This Agent orchestrates multiple tools to automate a recruitment analysis workflow:
# 1️⃣ Extract job description keywords
# 2️⃣ Scan PDF resumes for matching skills
# 3️⃣ Transcribe interview audio
# 4️⃣ Evaluate interview alignment with the job requirements
# The agent will return a consolidated evaluation report

recruiter_agent = Agent(
    name="AI Recruiter Assistant",
    instructions="""
You are helping a recruiter automate candidate evaluation:
1) Extract keywords from the job description.
2) Scan resumes for keyword matches.
3) Transcribe the interview file.
4) Evaluate how the interview aligns with the job description.

Return a full structured report with:
- Extracted job keywords
- Resume keyword matches
- Interview transcription summary & alignment feedback
""",
    tools=[
        extract_keywords_from_job_description,
        scan_resumes_for_keywords,
        transcribe_interview,
        analyze_interview_relevance
    ],
    model="gpt-4o",
    model_settings=ModelSettings(truncation="auto")
)

# ─────────────────────────────────────────────
# 8️⃣ Input data: Job Description + Interview Audio Path
# ─────────────────────────────────────────────

# Job description used for extracting keywords and relevance analysis
JOB_DESCRIPTION = """
We're hiring a full-stack engineer with experience in React, Python, REST APIs,
and deployment on cloud platforms like AWS.
"""

# Resolve the audio file path where the generated interview MP3 is stored
CURRENT_DIR = Path.cwd()
INTERVIEW_AUDIO_PATH = CURRENT_DIR / "audio_interview" / "audio_interview.mp3"

print("Audio file path set to:", INTERVIEW_AUDIO_PATH)


# ─────────────────────────────────────────────
# 9️⃣ Execute full Recruiter AI pipeline workflow
# ─────────────────────────────────────────────

# Prompt instructing the AI agent to:
# 1) Extract job keywords
# 2) Scan resumes for matches
# 3) Transcribe the interview audio
# 4) Analyze interview relevance

prompt = f"""
Please process this job description:

{JOB_DESCRIPTION}

Then scan local resumes for matches.
Finally, process the audio file here: {INTERVIEW_AUDIO_PATH}
and review whether the questions align with the job description.
"""

# Run the pipeline via the Agent Runner
result = await Runner.run(recruiter_agent, input=prompt)

# Display the final output report
print(result.final_output)